Factors (peptide levels) significantly affecting UPDRS_1

1. Load the data
2. Merge the data (peptide and clinical)
3. Process the data (impute null values, encoding)
4. Build the model
5. Coefficient Analysis (Identidy the top 10 variables)

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns # visualization
from sklearn.linear_model import LinearRegression # for model


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
file_list = {}
for dirname, _, filenames in os.walk('/kaggle/input'):
    print(dirname, _, filenames)
    for filename in filenames:
        file_list[filename.split('.')[0]] = os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input ['amp-parkinsons-disease-progression-prediction'] []
/kaggle/input/amp-parkinsons-disease-progression-prediction ['amp_pd_peptide', 'example_test_files'] ['train_proteins.csv', 'train_clinical_data.csv', 'public_timeseries_testing_util.py', 'supplemental_clinical_data.csv', 'train_peptides.csv']
/kaggle/input/amp-parkinsons-disease-progression-prediction/amp_pd_peptide [] ['competition.cpython-37m-x86_64-linux-gnu.so', '__init__.py']
/kaggle/input/amp-parkinsons-disease-progression-prediction/example_test_files [] ['sample_submission.csv', 'test_proteins.csv', 'test_peptides.csv', 'test.csv']


In [2]:
# list of files #
file_list

{'train_proteins': '/kaggle/input/amp-parkinsons-disease-progression-prediction/train_proteins.csv',
 'train_clinical_data': '/kaggle/input/amp-parkinsons-disease-progression-prediction/train_clinical_data.csv',
 'public_timeseries_testing_util': '/kaggle/input/amp-parkinsons-disease-progression-prediction/public_timeseries_testing_util.py',
 'supplemental_clinical_data': '/kaggle/input/amp-parkinsons-disease-progression-prediction/supplemental_clinical_data.csv',
 'train_peptides': '/kaggle/input/amp-parkinsons-disease-progression-prediction/train_peptides.csv',
 'competition': '/kaggle/input/amp-parkinsons-disease-progression-prediction/amp_pd_peptide/competition.cpython-37m-x86_64-linux-gnu.so',
 '__init__': '/kaggle/input/amp-parkinsons-disease-progression-prediction/amp_pd_peptide/__init__.py',
 'sample_submission': '/kaggle/input/amp-parkinsons-disease-progression-prediction/example_test_files/sample_submission.csv',
 'test_proteins': '/kaggle/input/amp-parkinsons-disease-progres

In [3]:
## Load the data ##
train_proteins = pd.read_csv(file_list['train_proteins'])
train_peptides = pd.read_csv(file_list['train_peptides'])
train_clinical_data = pd.read_csv(file_list['train_clinical_data'])
train_peptides.head()

,visit_id,visit_month,patient_id,UniProt,Peptide,PeptideAbundance
0,55_0,0,55,O00391,NEQEQPLGQWHLS,11254.3
1,55_0,0,55,O00533,GNPEPTFSWTK,102060.0
2,55_0,0,55,O00533,IEIPSSVQQVPTIIK,174185.0
3,55_0,0,55,O00533,KPQSAVYSTGSNGILLC(UniMod_4)EAEGEPQPTIK,27278.9
4,55_0,0,55,O00533,SMEQNGPGLEYR,30838.7


In [4]:
# peptide level data
peptide_df = train_peptides.pivot(index = 'visit_id', columns = 'Peptide')['PeptideAbundance']
print('peptide data',peptide_df.head())

# clinical level data

train_clinical_data.index = train_clinical_data['visit_id']
print('clinical data',train_clinical_data.head())

## merge the cinical nad peptide data set ##
train_x_y = pd.merge(train_clinical_data[['visit_month','updrs_1','upd23b_clinical_state_on_medication']],peptide_df, right_index = True, left_index= True)
train_x_y.head()

peptide data Peptide   AADDTWEPFASGK  AAFGQGSGPIMLDEVQC(UniMod_4)TGTEASLADC(UniMod_4)K  \
visit_id                                                                    
10053_0       6580710.0                                           31204.4   
10053_12      6333510.0                                           52277.6   
10053_18      7129640.0                                           61522.0   
10138_12      7404780.0                                           46107.2   
10138_24     13788300.0                                           56910.3   

Peptide   AAFTEC(UniMod_4)C(UniMod_4)QAADK  AANEVSSADVK  \
visit_id                                                  
10053_0                          7735070.0          NaN   
10053_12                         5394390.0          NaN   
10053_18                         7011920.0      35984.7   
10138_12                        10610900.0          NaN   
10138_24                         6906160.0      13785.5   

Peptide   AATGEC(UniMod_4)TATVGKR

,visit_month,updrs_1,upd23b_clinical_state_on_medication,AADDTWEPFASGK,AAFGQGSGPIMLDEVQC(UniMod_4)TGTEASLADC(UniMod_4)K,AAFTEC(UniMod_4)C(UniMod_4)QAADK,AANEVSSADVK,AATGEC(UniMod_4)TATVGKR,AATVGSLAGQPLQER,AAVYHHFISDGVR,...,YSLTYIYTGLSK,YTTEIIK,YVGGQEHFAHLLILR,YVM(UniMod_35)LPVADQDQC(UniMod_4)IR,YVMLPVADQDQC(UniMod_4)IR,YVNKEIQNAVNGVK,YWGVASFLQK,YYC(UniMod_4)FQGNQFLR,YYTYLIMNK,YYWGGQYTWDMAK
visit_id,,,,,,,,,,,,,,,,,,,,,
55_0,0,10.0,NaN,8984260.0,53855.6,8579740.0,NaN,19735.4,114400.0,46371.1,...,201158.0,16492.3,3810270.0,106894.0,580667.0,131155.0,165851.0,437305.0,46289.2,14898.4
55_6,6,8.0,NaN,8279770.0,45251.9,8655890.0,49927.5,23820.4,90539.4,38652.4,...,171079.0,13198.8,4119520.0,113385.0,514861.0,103512.0,144607.0,457891.0,40047.7,20703.9
55_12,12,10.0,On,8382390.0,53000.9,8995640.0,45519.2,17813.5,147312.0,45840.9,...,231772.0,17873.8,5474140.0,116286.0,711815.0,136943.0,181763.0,452253.0,54725.1,21841.1
55_36,36,17.0,On,10671500.0,58108.4,9985420.0,52374.0,19373.3,64356.1,49793.2,...,185290.0,18580.5,2659660.0,90936.9,679163.0,128593.0,203680.0,498621.0,52792.7,13973.7
942_6,6,8.0,NaN,6177730.0,42682.6,3596660.0,25698.8,17130.6,86471.5,41007.9,...,226314.0,6399.8,NaN,57571.4,480951.0,80001.2,79661.9,573300.0,48005.8,15674.1


In [5]:
## Treat null values ##
train_x_y['upd23b_clinical_state_on_medication'] = train_x_y['upd23b_clinical_state_on_medication'].fillna('NA')

# imputing 0 for all the peptide columns where we see null values
for col in train_x_y.columns:
    if col not in ['upd23b_clinical_state_on_medication', 'visit_month', 'updrs_1']:
        train_x_y[col] = train_x_y[col].fillna(0)
        
# encoding ##
to_map = {'NA': 0, 'Off':1,'On': 2}

train_x_y['upd23b_clinical_state_on_medication'] = train_x_y['upd23b_clinical_state_on_medication'].replace(to_map)

# reset the index #
train_x_y.reset_index(inplace = True, drop = True)
train_x_y.head()

,visit_month,updrs_1,upd23b_clinical_state_on_medication,AADDTWEPFASGK,AAFGQGSGPIMLDEVQC(UniMod_4)TGTEASLADC(UniMod_4)K,AAFTEC(UniMod_4)C(UniMod_4)QAADK,AANEVSSADVK,AATGEC(UniMod_4)TATVGKR,AATVGSLAGQPLQER,AAVYHHFISDGVR,...,YSLTYIYTGLSK,YTTEIIK,YVGGQEHFAHLLILR,YVM(UniMod_35)LPVADQDQC(UniMod_4)IR,YVMLPVADQDQC(UniMod_4)IR,YVNKEIQNAVNGVK,YWGVASFLQK,YYC(UniMod_4)FQGNQFLR,YYTYLIMNK,YYWGGQYTWDMAK
0,0,10.0,0,8984260.0,53855.6,8579740.0,0.0,19735.4,114400.0,46371.1,...,201158.0,16492.3,3810270.0,106894.0,580667.0,131155.0,165851.0,437305.0,46289.2,14898.4
1,6,8.0,0,8279770.0,45251.9,8655890.0,49927.5,23820.4,90539.4,38652.4,...,171079.0,13198.8,4119520.0,113385.0,514861.0,103512.0,144607.0,457891.0,40047.7,20703.9
2,12,10.0,2,8382390.0,53000.9,8995640.0,45519.2,17813.5,147312.0,45840.9,...,231772.0,17873.8,5474140.0,116286.0,711815.0,136943.0,181763.0,452253.0,54725.1,21841.1
3,36,17.0,2,10671500.0,58108.4,9985420.0,52374.0,19373.3,64356.1,49793.2,...,185290.0,18580.5,2659660.0,90936.9,679163.0,128593.0,203680.0,498621.0,52792.7,13973.7
4,6,8.0,0,6177730.0,42682.6,3596660.0,25698.8,17130.6,86471.5,41007.9,...,226314.0,6399.8,0.0,57571.4,480951.0,80001.2,79661.9,573300.0,48005.8,15674.1


In [6]:
## train - test split ##

train_x = train_x_y.copy()
train_x = train_x.drop('updrs_1', axis=1)

train_y = train_x_y['updrs_1']

## Model Building ##

model = LinearRegression()
model.fit(train_x, train_y)
model.score(train_x, train_y)

0.9456793475177111

In [7]:
# Coefficient Analysis 
# store the coefficient and column names in the dataframe #
peptide_coefficent_df = pd.DataFrame({'column_name': train_x.columns, 'coefficient': model.coef_})
peptide_coefficent_df = peptide_coefficent_df.sort_values(ascending = False, by = 'coefficient')
peptide_coefficent_df.head(10)

,column_name,coefficient
1,upd23b_clinical_state_on_medication,2.045871
689,RTHLPEVFLSK,0.030061
0,visit_month,0.009166
335,HNC(UniMod_4)MENSIC(UniMod_4)R,0.001225
680,RLEGQEEEEDNRDSSMK,0.001223
818,TM(UniMod_35)QENSTPRED,0.000553
868,VFAVSHGR,0.000449
914,VTEPISAESGEQVER,0.000408
384,INENTGSVSVTR,0.000377
643,QFTSSTSYNR,0.000368


# findings
1. Patient who consume medication have an impact on UPDRS score followed by peptide levels (RTHLPEVFLSK, Visit_month)

1. This is an initial analysis to understand the factors which is affecting the UPDRS_1 (rating scale of 1) and peptide levels at various timestamp
2. As a next step, I will continue my analysis with UPDRS_2,3,4
3. And also I will include protein levels in my next notebook